In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
# import pandas_datareader.data as web
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
from functools import partial
import talib
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm, kurtosis, skew

In [ ]:


#data = pd.read_csv('/Users/guneykan/Downloads/48_Industry_Portfolios.CSV', index_col=0)
data_daily = pd.read_csv('/Users/frank/Downloads/48_Industry_Portfolios_Daily.csv', index_col=0)
data_monthly = pd.read_csv('/Users/frank/Downloads/48_Industry_Portfolios.csv', index_col=0)

data_daily = data_daily[('197401' <= data_daily.index.values.astype(str)) & (data_daily.index.values.astype(str) <= '201912')]
data_monthly = data_monthly[('197401' <= data_monthly.index.values.astype(str)) & (data_monthly.index.values.astype(str) <= '201912')]


data_daily.index = pd.to_datetime(data_daily.index, format="%Y%m%d")
data_monthly.index = pd.to_datetime(data_monthly.index, format="%Y%m")

date_daily = data_daily.index.date
date_monthly = data_monthly.index.date

# The rate of return matrix, fill missing values with 100%
R = data_daily.values.T.astype(float)
R[R < -99]
R = (R + 100) / 100  # Gross Return

In [ ]:
data_daily.head()

# Daily

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

class BanditPortfolio:
    def __init__(self, R):
        self.R = R
        self.n_arms, self.n_samples = R.shape
        
    def UCB(self, window_size):
        self.reward = np.ones(self.n_samples - window_size)
        self.played_times = np.ones(self.n_arms)
        
        for t in range(window_size, self.n_samples):
            sliceR = self.R[:, t-window_size:t]
            
            # Compute the covariance matrix
            covariance_matrix =  np.cov(sliceR)
            
            # Eigenvalue Decomposition
            A, H = np.linalg.eig(covariance_matrix)
            
            # All eigenvalues are positive
            assert(np.sum(A<0) == 0)
            
            # Sort the eigenvalues
            idx = np.argsort(-A)
            A = np.diag(A[idx])
            H = H[:,idx]
            l = np.argwhere(np.median(np.diag(A)) > np.diag(A))[0][0]
            
            # Normalized weight
            H /= np.sum(H, axis= 0)
            ANew =  H.T.dot(covariance_matrix).dot(H)
            
            # Compute the sharpe ratio
            portfolio_reward = H.T.dot(sliceR)

            sharpe_ratio = np.mean(portfolio_reward, axis=1) / np.sqrt(ANew.diagonal())
            #sharpe_ratio = MinMaxScaler().fit_transform(sharpe_ratio.reshape(-1,1)).reshape(-1)
            
            # Compute the upper bound of expected reward
            sharpe_ratio_upper_bound = sharpe_ratio + np.sqrt((2*np.log(t))/(window_size+self.played_times))
            
            # Select the optimal arm
            action1 = np.argmax(sharpe_ratio_upper_bound[:l])
            action2 = np.argmax(sharpe_ratio_upper_bound[l:])+l

            self.played_times[action1] += 1
            self.played_times[action2] += 1

            # Optimal weight
            Adiag = ANew.diagonal()
            theta = Adiag[action1] / (Adiag[action1] + Adiag[action2])
            self.weight = (1-theta)*H[:,action1] + theta*H[:,action2]
            
            self.reward[t-window_size] = self.weight.dot(self.R[:,t])
            
    def TS(self, window_size):
        self.reward = np.ones(self.n_samples - window_size)
        self.mv_reward = np.ones(self.n_samples - window_size)
        self.played_times = np.ones(self.n_arms)
        self.success = np.ones(4)
        self.fail = np.ones(4)

        for t in range(window_size, self.n_samples):

            sliceR = self.R[:, t-window_size:t]

            # Compute the covariance matrix
            covariance_matrix = np.cov(sliceR)  # histroical covariance

            # Eigenvalue Decomposition
            A, H = np.linalg.eig(covariance_matrix)  # equation 5

            # All eigenvalues are positive
            assert(np.sum(A < 0) == 0)

            # Sort the eigenvalues
            idx = np.argsort(-A)  # sort eigenvalues
            A = np.diag(A[idx])  #  eigenvalues as vector
            H = H[:, idx]  #  n(number of assets) orthonormal portfolios
            l = np.argwhere(np.median(np.diag(A)) > np.diag(A))[0][0]

            # Normalized weight
            H /= np.sum(H, axis=0)  # equation 7 normalized eigenvectors
            #  equation 8 normalized eigenvalues matrix
            ANew = H.T.dot(covariance_matrix).dot(H)

            # Compute the sharpe ratio
            #  Return of each port. in the slice window
            portfolio_reward = H.T.dot(sliceR)
            # Estimator of portfolio return, rolling average
            sharpe_ratio = np.mean(
                portfolio_reward, axis=1) / np.sqrt(ANew.diagonal())
            # Normalize the sharpe ratio
            #sharpe_ratio = MinMaxScaler().fit_transform(
            #    sharpe_ratio.reshape(-1, 1)).reshape(-1)

            # Compute the upper bound of expected reward
            sharpe_ratio_upper_bound = sharpe_ratio + \
                np.sqrt((2*np.log(t))/(window_size+self.played_times))

            # Select the optimal arm
            #  passive portfolios
            action1 = np.argmax(sharpe_ratio_upper_bound[:l])
            #  active portfolios
            action2 = np.argmax(sharpe_ratio_upper_bound[l:])+l

            #  update the times portfolio played
            self.played_times[action1] += 1
            self.played_times[action2] += 1  # update the second action

            # Optimal weight, Min var allocation between 2 chosen portfolios
            Adiag = ANew.diagonal()
            theta = Adiag[action1] / (Adiag[action1] + Adiag[action2])
            
            self.psr_set = []
            #psr_second_set = []
            for a in range(len(sharpe_ratio)):
                sr = sharpe_ratio[a]
                n = (window_size+self.played_times[a])/window_size
                skewness = skew(portfolio_reward[a, :])
                kurto = kurtosis(portfolio_reward[a, :])
                nomin = (sr-np.mean(sharpe_ratio))*np.sqrt(n)
                denom = np.sqrt(np.abs((1 + 0.5*sr**2 - skewness*sr + ((kurto-3)/4)*sr**2))/(n-1))
                self.psr_set.append(nomin/denom)
                
                
            self.psr_set = np.array([norm.cdf(a) for a in self.psr_set])
            
            sharpe_ratio_upper_bound_psr = (sharpe_ratio + \
                np.sqrt((2*np.log(t))/(window_size+self.played_times)))*np.array(self.psr_set)
                
            
            action_1_1 = np.argmax(sharpe_ratio_upper_bound_psr[:l])
            action_1_2 = np.argmax(sharpe_ratio_upper_bound_psr[l:])+l


            # Optimal weight
            theta_ = Adiag[action_1_1] / (Adiag[action_1_1] + Adiag[action_1_2])
                        
            
            
            self.weight_1 = (1-theta)*H[:, action1] + theta*H[:, action2]
            
            self.weight_2 = np.ones(self.n_arms)/self.n_arms
            
            self.weight_3 = (np.linalg.inv(covariance_matrix)@np.ones(self.n_arms).reshape(-1, 1))/(np.ones(
                self.n_arms).reshape(-1, 1).T@np.linalg.inv(covariance_matrix)@np.ones(self.n_arms).reshape(-1, 1))
            
            self.weight_4 = (1-theta_)*H[:,action_1_1] + theta_*H[:,action_1_2]
            
            
            final_actions = [self.weight_1, self.weight_2, self.weight_3, self.weight_4]
            draws = [np.random.beta(self.success[action], self.fail[action])
                     for action in range(4)]
            
            final_weight = final_actions[np.argmax(draws)]
            
            self.reward[t-window_size] = final_weight.T.dot(self.R[:, t])
            self.mv_reward[t-window_size] = self.weight_3.T.dot(self.R[:, t])
            
            rewards = [self.weight_1.T.dot(self.R[:, t]), self.weight_2.T.dot(self.R[:, t]), self.weight_3.T.dot(self.R[:, t]), self.weight_4.T.dot(self.R[:, t])]
            
            

            if np.max(rewards) == final_weight.T@self.R[:, t]:
                self.success[np.argmax(draws)] += 1
                other_idx = [a for a in range(4) if a != np.argmax(rewards)]
                self.fail[other_idx[0]] +=1
                self.fail[other_idx[1]] +=1
                self.fail[other_idx[2]] +=1
            else:
                other_idx = [a for a in range(4) if (a != np.argmax(rewards)) & (a != np.argmax(draws))]
                self.success[np.argmax(rewards)] +=1
                self.fail[other_idx[0]] +=1
                self.fail[other_idx[1]] +=1
                self.fail[np.argmax(draws)] += 1
                
    def UCBPSR(self, window_size):
        self.reward = np.ones(self.n_samples - window_size)
        self.played_times = np.ones(self.n_arms)
        self.psr = {}
        
        for t in range(window_size, self.n_samples):
            sliceR = self.R[:, t-window_size:t]
            
            # Compute the covariance matrix
            covariance_matrix =  np.cov(sliceR)
            
            # Eigenvalue Decomposition
            A, H = np.linalg.eig(covariance_matrix)
            
            # All eigenvalues are positive
            assert(np.sum(A<0) == 0)
            
            # Sort the eigenvalues
            idx = np.argsort(-A)
            A = np.diag(A[idx])
            H = H[:,idx]
            l = np.argwhere(np.median(np.diag(A)) > np.diag(A))[0][0]
            
            # Normalized weight
            H /= np.sum(H, axis= 0)
            ANew =  H.T.dot(covariance_matrix).dot(H)
            
            # Compute the sharpe ratio
            portfolio_reward = H.T.dot(sliceR)

            
            sharpe_ratio = (np.mean(portfolio_reward, axis=1) / np.sqrt(ANew.diagonal()))
            #sharpe_ratio = MinMaxScaler().fit_transform(sharpe_ratio.reshape(-1,1)).reshape(-1)
            #scl = MinMaxScaler()
            #sharpe_ratio = scl.fit_transform(sharpe_ratio.reshape(-1,1)).reshape(-1)
            
            
            
            self.psr_set = []
            for a in range(len(sharpe_ratio)):
                sr = sharpe_ratio[a]
                n = (window_size+self.played_times[a])/window_size
                skewness = skew(portfolio_reward[a, :])
                kurto = kurtosis(portfolio_reward[a, :])
                nomin = (sr-np.mean(sharpe_ratio))*np.sqrt(n)
                denom = np.sqrt(np.abs((1 + 0.5*sr**2 - skewness*sr + ((kurto-3)/4)*sr**2))/(n-1))
                #psr = norm.cdf(nomin/denom)
                self.psr_set.append(nomin/denom)
            #self.psr_set = scl.fit_transform(np.array(self.psr_set).reshape(-1,1)).reshape(-1)
            self.psr_set = np.array([norm.cdf(a) for a in self.psr_set])
            self.psr[t] = self.psr_set
            
            # Compute the upper bound of expected reward
            #sharpe_ratio_upper_bound = sharpe_ratio + np.sqrt((2*np.log(t))/(window_size+self.played_times))
            sharpe_ratio_upper_bound = (sharpe_ratio + \
                np.sqrt((2*np.log(t))/(window_size+self.played_times)))*np.array(self.psr_set)
            
            action1 = np.argmax(sharpe_ratio_upper_bound[:l])
            action2 = np.argmax(sharpe_ratio_upper_bound[l:])+l
            
            # Select the optimal arm
            #action1 = np.argmax(self.psr_set[:l])
            #action2 = np.argmax(self.psr_set[l:])+l

            self.played_times[action1] += 1
            self.played_times[action2] += 1

            # Optimal weight
            Adiag = ANew.diagonal()
            theta = Adiag[action1] / (Adiag[action1] + Adiag[action2])
            self.weight = (1-theta)*H[:,action1] + theta*H[:,action2]
            
            self.reward[t-window_size] = self.weight.dot(self.R[:,t])
    def PSR(self, window_size):
        self.reward = np.ones(self.n_samples - window_size)
        self.played_times = np.ones(self.n_arms)
        self.psr = {}
        
        for t in range(window_size, self.n_samples):
            sliceR = self.R[:, t-window_size:t]
            
            # Compute the covariance matrix
            covariance_matrix =  np.cov(sliceR)
            
            # Eigenvalue Decomposition
            A, H = np.linalg.eig(covariance_matrix)
            
            # All eigenvalues are positive
            assert(np.sum(A<0) == 0)
            
            # Sort the eigenvalues
            idx = np.argsort(-A)
            A = np.diag(A[idx])
            H = H[:,idx]
            l = np.argwhere(np.median(np.diag(A)) > np.diag(A))[0][0]
            
            # Normalized weight
            H /= np.sum(H, axis= 0)
            ANew =  H.T.dot(covariance_matrix).dot(H)
            
            # Compute the sharpe ratio
            portfolio_reward = H.T.dot(sliceR)

            
            sharpe_ratio = (np.mean(portfolio_reward, axis=1) / np.sqrt(ANew.diagonal()))
            scl = MinMaxScaler(feature_range=(-3,3))
            #sharpe_ratio = scl.fit_transform(sharpe_ratio.reshape(-1,1)).reshape(-1)
            
            self.psr_set = []
            #psr_second_set = []
            for a in range(len(sharpe_ratio)):
                sr = sharpe_ratio[a]
                n = (window_size+self.played_times[a])/window_size
                skewness = skew(portfolio_reward[a, :])
                kurto = kurtosis(portfolio_reward[a, :])
                nomin = (sr-np.mean(sharpe_ratio))*np.sqrt(n-1)
                denom = np.sqrt(np.abs(1-(skewness*sr) + ((kurto-1)/4)*(sr**2)))
                #psr = norm.cdf(nomin/denom)
                self.psr_set.append(nomin/denom)
                
            self.psr_set = np.array([norm.cdf(a) for a in self.psr_set])
            self.psr[t] = self.psr_set
            
            # Compute the upper bound of expected reward
            #sharpe_ratio_upper_bound = sharpe_ratio + np.sqrt((2*np.log(t))/(window_size+self.played_times))
            #sharpe_ratio_upper_bound = (sharpe_ratio + \
            #    np.sqrt((2*np.log(t))/(window_size+self.played_times)))*np.array(self.psr_set)
            
            #action1 = np.argmax(sharpe_ratio_upper_bound[:l])
            #action2 = np.argmax(sharpe_ratio_upper_bound[l:])+l
            
            # Select the optimal arm
            action1 = np.argmax(self.psr_set[:l])
            action2 = np.argmax(self.psr_set[l:])+l

            self.played_times[action1] += 1
            self.played_times[action2] += 1

            # Optimal weight
            Adiag = ANew.diagonal()
            theta = Adiag[action1] / (Adiag[action1] + Adiag[action2])
            self.weight = (1-theta)*H[:,action1] + theta*H[:,action2]
            
            self.reward[t-window_size] = self.weight.dot(self.R[:,t])

In [ ]:
window_size = 120
orthogonal_bandit_portfolio = BanditPortfolio(R)

In [ ]:
orthogonal_bandit_portfolio.UCB(window_size = window_size)
ucb_ret = orthogonal_bandit_portfolio.reward

In [ ]:
orthogonal_bandit_portfolio.TS(window_size = window_size)
ts_ret = orthogonal_bandit_portfolio.reward
mv_ret = orthogonal_bandit_portfolio.mv_reward

In [ ]:
orthogonal_bandit_portfolio.PSR(window_size = window_size)
psr_ret = orthogonal_bandit_portfolio.reward

In [ ]:
orthogonal_bandit_portfolio.UCBPSR(window_size = window_size)
ucbpsr_ret = orthogonal_bandit_portfolio.reward

In [ ]:
#Baseline
constant_weight_rebalance = np.cumprod(R[:,window_size:].mean(axis=0))
equal_weight_portfolio = np.mean(np.cumprod(R[:,window_size:], axis=1), axis=0)b

date = data_daily.index.values[window_size:]

In [ ]:
ew_ret = np.mean(R[:,window_size:], axis=0)

In [ ]:
mv_ret.shape

In [8]:

all_rets = {"MVP": mv_ret, "EW": ew_ret, "UCB1": ucb_ret, "TS": ts_ret, "MaxPSR": psr_ret, "PW-UCB1": ucbpsr_ret} 

NameError: name 'mv_ret' is not defined

In [ ]:
z = pd.DataFrame(all_rets)-1

In [ ]:
z.to_csv("new_rets.csv")

In [ ]:

all_wealth = {"MVP": mv_wealth, "CWR": constant_weight_rebalance, "EW": equal_weight_portfolio, "UCB1": ucb_wealth, "TS": ts_wealth, "MaxPSR": psr_wealth, "PW-UCB1": ucbpsr_wealth} 

In [ ]:
results = pd.DataFrame(all_wealth)
results_pct = results.pct_change().dropna()

In [ ]:
results_pct

In [ ]:
results_pct.std()*100*np.sqrt(252)

In [ ]:
results.index = date

In [ ]:
results_pct.index = date[1:]

In [ ]:
years = pd.date_range(start="1974", end="2019", freq="y").year
years = [str(a) for a in years]

In [ ]:
yearly_sharpes = {}

for col in results_pct.columns:
    col_ = {}
    for a in range(1, len(years)):
        col_[years[a]] = np.sqrt(252)*np.mean(results_pct.loc[years[a-1]:years[a], col])/np.std(results_pct.loc[years[a-1]:years[a], col])
    yearly_sharpes[col] = col_

In [ ]:
yearly_sharpes = pd.DataFrame(yearly_sharpes)

In [ ]:
results_pct.to_csv("pct_changes.csv")

In [ ]:
yearly_sharpes.mean().sort_values(ascending=False)

In [ ]:
((np.mean(results_pct)/np.std(results_pct))*np.sqrt(252)).sort_values(ascending=False)

In [ ]:
results.loc["1984":"1994", :]

In [ ]:
results.to_csv("results.csv")

In [ ]:
wealth_74_84 = np.cumprod(pd.DataFrame(np.concatenate((np.ones(
    7).reshape(-1, 7), results_pct.loc["1974":"1984", :].values+1), axis=0), columns=results_pct.columns, index=pd.date_range(start=results_pct.loc["1974":"1984", :].index[0]-pd.Timedelta("1d"), end=results_pct.loc["1974":"1984", :].index[-1], periods=results_pct.loc["1974":"1984", :].shape[0]+1).date))

wealth_84_94 = np.cumprod(pd.DataFrame(np.concatenate((np.ones(
    7).reshape(-1, 7), results_pct.loc["1984":"1994", :].values+1), axis=0), columns=results_pct.columns, index=pd.date_range(start=results_pct.loc["1984":"1994", :].index[0]-pd.Timedelta("1d"), end=results_pct.loc["1984":"1994", :].index[-1], periods=results_pct.loc["1984":"1994", :].shape[0]+1).date))

wealth_94_04 = np.cumprod(pd.DataFrame(np.concatenate((np.ones(
    7).reshape(-1, 7), results_pct.loc["1994":"2004", :].values+1), axis=0), columns=results_pct.columns, index=pd.date_range(start=results_pct.loc["1994":"2004", :].index[0]-pd.Timedelta("1d"), end=results_pct.loc["1994":"2004", :].index[-1], periods=results_pct.loc["1994":"2004", :].shape[0]+1).date))

wealth_04_14 = np.cumprod(pd.DataFrame(np.concatenate((np.ones(
    7).reshape(-1, 7), results_pct.loc["2004":"2014", :].values+1), axis=0), columns=results_pct.columns, index=pd.date_range(start=results_pct.loc["2004":"2014", :].index[0]-pd.Timedelta("1d"), end=results_pct.loc["2004":"2014", :].index[-1], periods=results_pct.loc["2004":"2014", :].shape[0]+1).date))

wealth_14_20 = np.cumprod(pd.DataFrame(np.concatenate((np.ones(
    7).reshape(-1, 7), results_pct.loc["2014":"2020", :].values+1), axis=0), columns=results_pct.columns, index=pd.date_range(start=results_pct.loc["2014":"2020", :].index[0]-pd.Timedelta("1d"), end=results_pct.loc["2014":"2020", :].index[-1], periods=results_pct.loc["2014":"2020", :].shape[0]+1).date))

In [ ]:
sns.set(style="whitegrid", font_scale=3.2)
plt.figure(figsize=(60, 25))
ax = sns.lineplot(data=results, palette="bright", linewidth=2.7, dashes=dash_styles)
ax.set(xlabel="Date", ylabel='Cum. Wealth', title="FF48 Algo. Comparison");
ax.figure.savefig("output.png")

In [ ]:
import seaborn as sns

dash_styles = ["",
               (4, 1.5),
               (1, 1),
               (3, 1, 1.5, 1),
               (5, 1, 1, 1),
               (5, 1, 2, 1, 2, 1),
               (2, 2, 3, 1.5),
               (1, 2.5, 3, 1.2)]



sns.set(style="whitegrid", font_scale=3.3)
plt.figure(figsize=(60, 100))

plt.subplot(5,1,1)
ax = sns.lineplot(data=wealth_74_84, palette="bright", linewidth=2.7, dashes=dash_styles)
ax.set(ylabel='Cum. Wealth', title="FF48 Algo. Comparison")

plt.subplot(5,1,2)
ax = sns.lineplot(data=wealth_84_94, palette="bright", linewidth=2.7, dashes=dash_styles)
ax.set(ylabel='Cum. Wealth')

In [ ]:
import seaborn as sns

dash_styles = ["",
               (4, 1.5),
               (1, 1),
               (3, 1, 1.5, 1),
               (5, 1, 1, 1),
               (5, 1, 2, 1, 2, 1),
               (2, 2, 3, 1.5),
               (1, 2.5, 3, 1.2)]



sns.set(style="whitegrid", font_scale=3.3)
plt.figure(figsize=(60, 100))

plt.subplot(5,1,1)
ax = sns.lineplot(data=wealth_74_84, palette="bright", linewidth=2.7, dashes=dash_styles)
ax.set(ylabel='Cum. Wealth', title="FF48 Algo. Comparison")

plt.subplot(5,1,2)
ax = sns.lineplot(data=wealth_84_94, palette="bright", linewidth=2.7, dashes=dash_styles)
ax.set(ylabel='Cum. Wealth')

plt.subplot(5,1,3)
ax = sns.lineplot(data=wealth_94_04, palette="bright", linewidth=2.7, dashes=dash_styles)
ax.set(ylabel='Cum. Wealth')

plt.subplot(5,1,4)
ax = sns.lineplot(data=wealth_04_14, palette="bright", linewidth=2.7, dashes=dash_styles)
ax.set(ylabel='Cum. Wealth')

plt.subplot(5,1,5)
ax = sns.lineplot(data=wealth_14_20, palette="bright", linewidth=2.7, dashes=dash_styles)
ax.set(xlabel="Date", ylabel='Cum. Wealth')
ax.figure.savefig("output_2.png")

In [ ]:
yearly_sharpes.to_csv("year_sharpes.csv")

# 10-Portfolios - Daily


In [ ]:

data_10 = pd.read_csv('/Users/guneykan/Downloads/48_Industry_Portfolios.CSV', index_col=0)

data_10 = data_10[('197401' <= data_10.index.values.astype(str)) & (data_10.index.values.astype(str) <= '201912')]

data_10.index = pd.to_datetime(data_10.index, format="%Y%m")

date_10 = data_10.index.date

# The rate of return matrix, fill missing values with 100%
R_10 = data_10.values.T.astype(float)
R_10[R_10 < -99]
R_10 = (R_10 + 100) / 100  # Gross Return

In [ ]:
window_size = 120
orthogonal_bandit_portfolio_10 = BanditPortfolio(R_10)

In [ ]:
orthogonal_bandit_portfolio_10.UCB(window_size = window_size)
ucb_10_wealth = np.cumprod(orthogonal_bandit_portfolio_10.reward)

In [ ]:
orthogonal_bandit_portfolio_10.UCBPSR(window_size = window_size)
ucbpsr_10_wealth = np.cumprod(orthogonal_bandit_portfolio_10.reward)

In [ ]:
orthogonal_bandit_portfolio_10.TS(window_size = window_size)
ts_10_wealth = np.cumprod(orthogonal_bandit_portfolio_10.reward)
mv_10_wealth = np.cumprod(orthogonal_bandit_portfolio_10.mv_reward)


In [ ]:
orthogonal_bandit_portfolio_10.PSR(window_size = window_size)
psr_10_wealth = np.cumprod(orthogonal_bandit_portfolio_10.reward)


In [ ]:
cw_10_wealth = np.cumprod(R_10[:,window_size:].mean(axis=0))
ew_10_wealth = np.mean(np.cumprod(R_10[:,window_size:], axis=1), axis=0)

In [ ]:
all_10_wealth = {"MVP": mv_10_wealth, "CWR": cw_10_wealth, "EW": ew_10_wealth, "UCB1": ucb_10_wealth, "TS": ts_10_wealth, "MaxPSR": psr_10_wealth, "PW-UCB1": ucbpsr_10_wealth}

results_10 = pd.DataFrame(all_10_wealth)
results_pct_10 = results_10.pct_change().dropna()

In [ ]:
results_10

In [ ]:
results_pct_10.index = date_10[window_size+1:]

In [ ]:
years = pd.date_range(start="1974", end="2019", freq="y").year
years = [str(a) for a in years]

results_pct_10.index = pd.to_datetime(results_pct_10.index)


In [ ]:
yearly_sharpes = {}

for col in results_pct_10.columns:
    col_ = {}
    for a in range(1, len(years)):
        col_[years[a]] = np.sqrt(12)*np.mean(results_pct_10.loc[years[a-1]:years[a], col])/np.std(results_pct_10.loc[years[a-1]:years[a], col])
    yearly_sharpes[col] = col_

In [ ]:
yearly_sharpes = pd.DataFrame(yearly_sharpes)
yearly_sharpes.mean().sort_values(ascending=False)

In [ ]:
import seaborn as sns

dash_styles = ["",
               (4, 1.5),
               (1, 1),
               (3, 1, 1.5, 1),
               (5, 1, 1, 1),
               (5, 1, 2, 1, 2, 1),
               (2, 2, 3, 1.5),
               (1, 2.5, 3, 1.2)]
sns.set(style="whitegrid", font_scale=2.5)
plt.figure(figsize=(40, 18))
ax = sns.lineplot(data=results_10, palette="bright", linewidth=2.7, dashes=dash_styles)
ax.set(xlabel='Date', ylabel='Cum. Wealth', title="FF10 Algo. Comparison")

# 5-Port. Daily

In [ ]:

data_30 = pd.read_csv('/Users/guneykan/Downloads/5_Industry_Portfolios_Daily.CSV', index_col=0)

data_30 = data_30[('197401' <= data_30.index.values.astype(str)) & (data_30.index.values.astype(str) <= '201912')]

data_30.index = pd.to_datetime(data_30.index, format="%Y%m%d")

date_30 = data_30.index.date

# The rate of return matrix, fill missing values with 100%
R_30 = data_30.values.T.astype(np.float)
#R_30[R_30 < -99]
R_30 = (R_30 + 100) / 100  # Gross Return

In [ ]:
window_size = 120
orthogonal_bandit_portfolio_30 = BanditPortfolio(R_30)

In [ ]:
orthogonal_bandit_portfolio_30.UCB(window_size = window_size)
ucb_30_wealth = np.cumprod(orthogonal_bandit_portfolio_30.reward)

In [ ]:
orthogonal_bandit_portfolio_30.UCBPSR(window_size = window_size)
ucbpsr_30_wealth = np.cumprod(orthogonal_bandit_portfolio_30.reward)

In [ ]:
orthogonal_bandit_portfolio_30.TS(window_size = window_size)
ts_30_wealth = np.cumprod(orthogonal_bandit_portfolio_30.reward)
mv_30_wealth = np.cumprod(orthogonal_bandit_portfolio_30.mv_reward)


In [ ]:
orthogonal_bandit_portfolio_30.PSR(window_size = window_size)
psr_30_wealth = np.cumprod(orthogonal_bandit_portfolio_30.reward)

In [ ]:
cw_30_wealth = np.cumprod(R_30[:,window_size:].mean(axis=0))
ew_30_wealth = np.mean(np.cumprod(R_30[:,window_size:], axis=1), axis=0)

In [ ]:
date_30 = data_30.index.values[window_size:]
all_30_wealth = {"MVP": mv_30_wealth, "CWR": cw_30_wealth, "EW": ew_30_wealth, "UCB1": ucb_30_wealth, "TS": ts_30_wealth, "MaxPSR": psr_30_wealth, "PW-UCB1": ucbpsr_30_wealth}

results_30 = pd.DataFrame(all_30_wealth)
results_pct_30 = results_30.pct_change().dropna()
results_30.index = date_30
results_pct_30.index = date_30[1:]

In [ ]:


years = pd.date_range(start="1974", end="2019", freq="y").year
years = [str(a) for a in years]

results_pct_30.index = pd.to_datetime(results_pct_30.index)

yearly_sharpes = {}

for col in results_pct_30.columns:
    col_ = {}
    for a in range(1, len(years)):
        col_[years[a]] = np.sqrt(252)*np.mean(results_pct_30.loc[years[a-1]:years[a], col])/np.std(results_pct_30.loc[years[a-1]:years[a], col])
    yearly_sharpes[col] = col_
    
yearly_sharpes = pd.DataFrame(yearly_sharpes)
yearly_sharpes.mean().sort_values(ascending=False)

In [ ]:
import seaborn as sns

dash_styles = ["",
               (4, 1.5),
               (1, 1),
               (3, 1, 1.5, 1),
               (5, 1, 1, 1),
               (5, 1, 2, 1, 2, 1),
               (2, 2, 3, 1.5),
               (1, 2.5, 3, 1.2)]
sns.set(style="whitegrid", font_scale=2.5)
plt.figure(figsize=(40, 18))
ax = sns.lineplot(data=results_30, palette="bright", linewidth=2.7, dashes=dash_styles)
ax.set(xlabel='Date', ylabel='Cum. Wealth', title="FF30 Algo. Comparison")